## 1. Execute imports

In [2]:
import pandas as pd
import numpy as np
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report

In [3]:
import warnings
warnings.filterwarnings("ignore")
data_glass = pd.read_csv('datasets/banknote-authentication.csv', delimiter=',')
X = pd.read_csv('datasets/banknote_formula_b.csv', delimiter=',')
y = data_glass['Class']
features = list(X.columns.values)
y = pd.DataFrame(y)

## KNN

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

### Information gain

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [KNeighborsClassifier()]}]

In [7]:
%%time
knn_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_info.fit(X_train, y_train)

Wall time: 2.72 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000002256416BB80>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [KNeighborsClassifier()]}])

In [8]:
print(cross_val_score(knn_info, X, y, cv=10).mean())

0.9875965302020523


In [6]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', KNeighborsClassifier())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=0.913, cfg={'selector__k': 10}
>acc=0.996, est=0.920, cfg={'selector__k': 10}
>acc=1.000, est=0.913, cfg={'selector__k': 10}
>acc=1.000, est=0.914, cfg={'selector__k': 10}
>acc=1.000, est=0.909, cfg={'selector__k': 10}
Accuracy: 0.999 (0.001)
Wall time: 7.74 s


### Variance threshold

In [7]:
from sklearn.feature_selection import VarianceThreshold

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'classifier': [KNeighborsClassifier()]}]

In [11]:
%%time
knn_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
knn_variance.fit(X_train, y_train)

Wall time: 183 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}],
             scoring='accuracy')

In [12]:
print(cross_val_score(knn_variance, X, y, cv=10).mean())

1.0


In [8]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

    search_space = [{'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=0.996, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', KNeighborsClassifier())])
Accuracy: 0.999 (0.001)
Wall time: 683 ms


### Chi-Square

In [9]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', KNeighborsClassifier())])

In [15]:
search_space = [{'classifier': [KNeighborsClassifier()]}]

In [16]:
%%time
knn_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_chi.fit(X_train, y_train)

Wall time: 187 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x0000022562F90040>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}])

In [17]:
print(cross_val_score(knn_chi, X, y, cv=10).mean())

0.9606315455410982


In [10]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', KNeighborsClassifier())])
    # define search space
    search_space = [{'classifier': [KNeighborsClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = KNeighborsClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=0.825, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', KNeighborsClassifier())])
>acc=0.996, est=0.836, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=0.820, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=0.832, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', KNeighborsClassifier())])
>acc=1.000, est=0.823, cfg=Pipeline(steps=[('sca

### Logistic Regression

### Information gain

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': ['all']},
                {'classifier': [LogisticRegression()]}]

In [21]:
%%time
logistic_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_info.fit(X_train, y_train)

Wall time: 2.16 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000002256416BB80>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [LogisticRegression()]}])

In [22]:
print(cross_val_score(logistic_info, X, y, cv=10).mean())

0.982502909129377


In [12]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', LogisticRegression())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.993, est=0.984, cfg={'selector__k': 10}
>acc=0.985, est=0.980, cfg={'selector__k': 10}
>acc=0.993, est=0.977, cfg={'selector__k': 10}
>acc=0.985, est=0.982, cfg={'selector__k': 10}
>acc=0.993, est=0.982, cfg={'selector__k': 10}
Accuracy: 0.990 (0.004)
Wall time: 8.6 s


### Variance threshold

In [19]:
from sklearn.feature_selection import VarianceThreshold

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

search_space = [{'classifier': [LogisticRegression()]}]

In [25]:
%%time
logistic_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
logistic_variance.fit(X_train, y_train)

Wall time: 262 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}],
             scoring='accuracy')

In [26]:
print(cross_val_score(logistic_variance, X, y, cv=10).mean())

0.994895800275045


In [13]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

    search_space = [{'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.993, est=0.988, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.985, est=0.990, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.993, est=0.989, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.985, est=0.993, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
>acc=0.993, est=0.990, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', LogisticRegression())])
Accuracy: 0.990 (0.004)
Wall time: 1.24 s


### Chi-Square

In [27]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', LogisticRegression())])

In [29]:
search_space = [{'classifier': [LogisticRegression()]}]

In [30]:
%%time
logistic_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_chi.fit(X_train, y_train)

Wall time: 194 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x0000022562F90040>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}])

In [31]:
print(cross_val_score(logistic_chi, X, y, cv=10).mean())

0.9628054585845763


In [14]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', LogisticRegression())])
    # define search space
    search_space = [{'classifier': [LogisticRegression()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = LogisticRegression()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.993, est=0.969, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', LogisticRegression())])
>acc=0.985, est=0.974, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', LogisticRegression())])
>acc=0.993, est=0.964, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', LogisticRegression())])
>acc=0.985, est=0.966, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', LogisticRegression())])
>acc=0.993, est=0.965, cfg=Pipeline(steps=[('scaler', Mi

## SVM

In [15]:
from sklearn.svm import SVC

### Information gain

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'selector__k': ['all']},
                {'classifier': [SVC(kernel='linear')]}]

In [35]:
%%time
svm_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_info.fit(X_train, y_train)

Wall time: 2 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000002256416BB80>)),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [SVC(kernel='linear')]}])

In [36]:
print(cross_val_score(svm_info, X, y, cv=10).mean())

0.9897968898762297


In [16]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=9)),
                 ('classifier', SVC(kernel='linear'))])
    # define search space
    search_space = [{'selector__k': [9]}, {'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.985, est=0.984, cfg={'selector__k': 9}
>acc=0.985, est=0.985, cfg={'selector__k': 9}
>acc=0.989, est=0.983, cfg={'selector__k': 9}
>acc=0.985, est=0.987, cfg={'selector__k': 9}
>acc=0.993, est=0.985, cfg={'selector__k': 9}
Accuracy: 0.988 (0.003)
Wall time: 7.6 s


### Variance threshold

In [37]:
from sklearn.feature_selection import VarianceThreshold

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'classifier': [SVC(kernel='linear')]}]

In [39]:
%%time
svm_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
svm_variance.fit(X_train, y_train)

Wall time: 130 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'classifier': [SVC(kernel='linear')]}],
             scoring='accuracy')

In [40]:
print(cross_val_score(svm_variance, X, y, cv=10).mean())

0.9963503649635037


In [17]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold(3)),
                 ('classifier', SVC(kernel='linear'))])

    search_space = [{'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.993, est=0.987, cfg=Pipeline(steps=[('selector', VarianceThreshold(threshold=3)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.985, est=0.988, cfg=Pipeline(steps=[('selector', VarianceThreshold(threshold=3)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.989, est=0.988, cfg=Pipeline(steps=[('selector', VarianceThreshold(threshold=3)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.985, est=0.992, cfg=Pipeline(steps=[('selector', VarianceThreshold(threshold=3)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.993, est=0.991, cfg=Pipeline(steps=[('selector', VarianceThreshold(threshold=3)),
                ('classifier', SVC(kernel='linear'))])
Accuracy: 0.989 (0.003)
Wall time: 560 ms


### Chi-Square

In [41]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', SVC())])

In [43]:
search_space = [{'classifier': [SVC()]}]

In [44]:
%%time
svm_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_chi.fit(X_train, y_train)

Wall time: 198 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x0000022562F90040>)),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()]}])

In [45]:
print(cross_val_score(svm_chi, X, y, cv=10).mean())

0.9912461652385487


In [18]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', SVC(kernel='linear'))])
    # define search space
    search_space = [{'classifier': [SVC(kernel='linear')]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = SVC(kernel='linear')
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.993, est=0.979, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.985, est=0.975, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.989, est=0.978, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.985, est=0.980, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', SVC(kernel='linear'))])
>acc=0.993, est=0.982, cfg=Pipeline(steps=[('scaler', Mi

## Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB

### Information gain

In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', GaussianNB())])

search_space = [{'selector__k': ['all']},
                {'classifier': [GaussianNB()]}]

In [49]:
%%time
nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

Wall time: 2.32 s


GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000002256416BB80>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [GaussianNB()]}])

In [50]:
print(cross_val_score(nb_info, X, y, cv=10).mean())

0.8498201629112451


In [20]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', GaussianNB())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.836, est=0.846, cfg={'selector__k': 10}
>acc=0.858, est=0.834, cfg={'selector__k': 10}
>acc=0.843, est=0.842, cfg={'selector__k': 10}
>acc=0.814, est=0.840, cfg={'selector__k': 10}
>acc=0.850, est=0.843, cfg={'selector__k': 10}
Accuracy: 0.840 (0.015)
Wall time: 6.98 s


### Variance threshold

In [51]:
from sklearn.feature_selection import VarianceThreshold

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

search_space = [{'classifier': [GaussianNB()]}]

In [53]:
nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}], scoring='accuracy')

In [54]:
print(cross_val_score(nb_variance, X, y, cv=10).mean())

0.8498201629112451


In [21]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

    search_space = [{'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.836, est=0.846, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.858, est=0.834, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.843, est=0.842, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.814, est=0.840, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
>acc=0.850, est=0.843, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', GaussianNB())])
Accuracy: 0.840 (0.015)
Wall time: 438 ms


### Chi-Square

In [55]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', GaussianNB())])

In [57]:
search_space = [{'classifier': [GaussianNB()]}]

In [58]:
%%time
nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

Wall time: 113 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x0000022562F90040>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}])

In [59]:
print(cross_val_score(nb_chi, X, y, cv=10).mean())

0.8498201629112451


In [22]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', GaussianNB())])
    # define search space
    search_space = [{'classifier': [GaussianNB()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = GaussianNB()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=0.836, est=0.846, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', GaussianNB())])
>acc=0.858, est=0.834, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', GaussianNB())])
>acc=0.843, est=0.842, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', GaussianNB())])
>acc=0.814, est=0.840, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', GaussianNB())])
>acc=0.850, est=0.843, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                (

## Random Forest

In [23]:
from sklearn.ensemble import RandomForestRegressor

In [8]:
### Information gain

from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'selector__k': ['all']},
                {'classifier': [RandomForestRegressor()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

print(cross_val_score(nb_info, X, y, cv=10).mean())

0.0939233870967742
0.09518054623655914


In [24]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', RandomForestRegressor())])

    search_space = [{'selector__k': [10]},
                {'classifier': [RandomForestRegressor()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = RandomForestRegressor()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', RandomForestRegressor())])

search_space = [{'classifier': [RandomForestRegressor()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print(cross_val_score(nb_chi, X, y, cv=10).mean())

In [ ]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', RandomForestRegressor())])
    # define search space
    search_space = [{'classifier': [RandomForestRegressor()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
    # execute search
    result = _info.fit(X_train, y_train)
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = RandomForestRegressor()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [MLPClassifier()]}]

nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

print(cross_val_score(nb_info, X, y, cv=10).mean())

1.0


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

0.9919760922458478


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Walter Ortiz\anaconda3\lib\site-pac

0.994895800275045


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [26]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k=10)),
                 ('classifier', MLPClassifier())])
    # define search space
    search_space = [{'selector__k': [10]}, {'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=0.998, cfg={'selector__k': 10}
>acc=1.000, est=0.999, cfg={'selector__k': 10}
>acc=1.000, est=1.000, cfg={'selector__k': 10}
>acc=1.000, est=0.999, cfg={'selector__k': 10}
>acc=1.000, est=0.999, cfg={'selector__k': 10}
Accuracy: 1.000 (0.000)
Wall time: 1min 48s


In [ ]:
### Variance threshold

from sklearn.feature_selection import VarianceThreshold

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', VarianceThreshold()),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

print(cross_val_score(nb_variance, X, y, cv=10).mean())

In [27]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix].loc[:, features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', MLPClassifier())])

    search_space = [{'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    select_indexes = np.arange(len(X_train.columns)).reshape(1, -1)
    select_indexes = result.best_estimator_.named_steps['selector'].transform(select_indexes)
    selected_features = X.iloc[:,select_indexes.reshape(-1)]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
>acc=1.000, est=1.000, cfg=Pipeline(steps=[('selector', VarianceThreshold()),
                ('classifier', MLPClassifier())])
Accuracy: 1.000 (0.000)
Wall time: 44.4 s


In [ ]:
### Chi-Square

from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', MLPClassifier())])

search_space = [{'classifier': [MLPClassifier()]}]

nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

print(cross_val_score(nb_chi, X, y, cv=10).mean())

In [28]:
%%time
features = list(X.columns.values)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
# enumerate splits
outer_results = list()
for train_ix, test_ix in kf.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix][features], X.iloc[test_ix][features]
    y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
    # define the model
    pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k=10)),
                 ('classifier', MLPClassifier())])
    # define search space
    search_space = [{'classifier': [MLPClassifier()]}]
    # define search
    _info = GridSearchCV(pipe, search_space, scoring='accuracy', cv=None, refit=True)
    # execute search
    result = _info.fit(X_train, y_train)
    
    selected = result.best_estimator_.named_steps['selector'].get_support(indices=True)
    selected_features = X.iloc[:,selected]
    x_t = X_test[selected_features.columns.values]
    
    clf = MLPClassifier()
    clf.fit(X_train[selected_features.columns.values], y_train)
    # evaluate the model
    acc = accuracy_score(y_test, clf.predict(x_t))
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_estimator_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))

>acc=1.000, est=0.989, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', MLPClassifier())])
>acc=1.000, est=0.989, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', MLPClassifier())])
>acc=1.000, est=0.987, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', MLPClassifier())])
>acc=1.000, est=0.985, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x000001AA93E00DC0>)),
                ('classifier', MLPClassifier())])
>acc=1.000, est=0.985, cfg=Pipeline(steps=[('scaler', MinMaxScaler()),
     